In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# the goal of this notebook is to simulate a 2 state random walk.
## Does it show 3 stages of diffusion when mean resting times diverge?

In [ ]:
def walkit(k0,k1,N,rest='exp'):
    """ realize a random walk 
    k0 is resting time parameter
    k1 is motion time parameter
    N is number of double phases.
    for exponential, put rest='exp'
    for power law, put rest='pow'
    and set k0 = (a,t0)
    a is the power law tail parameter
    t0 is the minimum truncation time
    """
    #N = 500 # number of double phases
    #k0 = 1.0 # mean resting time
    #k1 = 1.0 # mean motion time
    if rest=='exp':
        psi0 = np.random.exponential(scale=k0,size=N)
    elif rest=='pow':
        a,m = k0
        psi0 =(np.random.pareto(a, N) + 1) * m
    psi1 = np.random.exponential(scale=k1,size=N)
    t = np.empty(2*N).astype('float') # sequence of times
    t[0::2] = psi0
    t[1::2] = psi1
    t = t.cumsum() # sum the times
    t = np.array([0]+list(t[:-1])) # needs to start from 0
    s = np.empty(2*N).astype('int')# sequence of states
    s[0::2] = 0 # starts at rest
    s[1::2] = 1
    return t,s

def plotit(walk):
    """plot a random walk"""
    t,s = walk
    widths = np.diff(t)
    plt.bar(t[:-1],s[:-1]+1,widths,align='edge',edgecolor='black')

def xit(walk):
    #calculate displacement versus time on a grid... 
    dt = 1e-3 # sampling interval 
    t,s = walk # realize a walk
    tmax = t.max()
    ts = np.arange(0,tmax,dt) # set of sampling times
    xs = np.zeros_like(ts) # set of positions
    i = np.searchsorted(t,ts)+1 # s[i] is the state at each sampling time
    i = i[i<len(s)] # only include values within the physical range t..not all values of ts  
    u = 1.0 # velocity
    mask = s[i]==1 # positions in x where movement happens
    # mask of all positions in x where movement hapns
    xs = xs[:len(mask)]
    ts = ts[:len(mask)]
    xs[mask]=u*dt
    xs = xs.cumsum()
    return ts,xs

def ensit(k0,k1,N,rest='exp',n=100,cutoff=None):
    """generate an ensemble of n walkers using walkfn
    formed from a particular parameter choice in walkit 
    cufoff is the maximum time to account for"""
    # generate an ensemble of walks
    tmax=0
    t = []
    disps = []
    for i in range(n):
        walk = walkit(k0,k1,N,rest=rest)
        if cutoff:
            ts,xs = xit(walk)
            xs = xs[ts<cutoff] # clip the arrays at the cutoff time 
            ts = ts[ts<cutoff] # this prevents memory explosion
            if len(ts)>len(t): # keep only the longest time array
                t = ts
            disps.append(xs)
        else:
            ts,xs = xit(walk)
            if len(ts)>len(t): # keep only the longest time array
                t = ts
            disps.append(xs)            
    disps = np.array(disps) # this makes a jagged array since the random walk simulations
    # will all have different lengths 
    # now post process
    # add nans to all arrays to make the jagged array square
    def numpy_fillna(data):
        # Get lengths of each row of data
        lens = np.array([len(i) for i in data])
        # Mask of valid places in each row
        mask = np.arange(lens.max()) < lens[:,None]
        # Setup output array and put elements from data into masked positions
        out = np.ones(mask.shape)*np.nan
        out[mask] = np.concatenate(data)
        return out
    disps = numpy_fillna(disps) # jagged array is now square
    return t, disps

def meanit(t, disps):
    val = np.nanmean(disps,0)
    mask = ~np.isnan(disps) # only include points wiht at least 50 samples
    mask = mask.sum(0)>50 # require at least 50 to not be nan in each time point
    return t[mask],val[mask]

def varit(t,disps):
    val = np.nanvar(disps,0)
    mask = ~np.isnan(disps) # only include points wiht at least 50 samples
    mask = mask.sum(0)>50 # require at least 50 to not be nan in each time point
    return t[mask],val[mask]

In [ ]:
walk = walkit(1,1,10)
plotit(walk)

In [ ]:
t,disps = ensit(1,1,100,n=100,cutoff= 100)
tv,var = varit(t,disps)
plt.loglog(tv,var,color='purple')
plt.xlim(1e-3)
plt.ylim(1e-6)

t,disps = ensit(0.1,1,100,n=100,cutoff=100)
tv,var = varit(t,disps)
plt.loglog(tv,var,color='blue')
plt.xlim(1e-3)
plt.ylim(1e-6)

# ok. exponential case works fine now. Need to implement power law case.

In [ ]:
a, m = 3., 2.  # shape and mode
s = (np.random.pareto(a, 1000) + 1) * m

In [ ]:
_=plt.hist(s,bins=50)

In [ ]:
%memit 
t,disps = ensit((1.5,3),1,100,n=100,rest='pow',cutoff=100)
tv,var = varit(t,disps)
plt.loglog(tv,var,color='purple')
plt.xlim(1e-3)
plt.ylim(1e-6)

# you stopped here. 
# got to add a maximum time in ensit. the thing explodes with power-law tails.